<a href="https://colab.research.google.com/github/chengdh/tool-ahead-of-time/blob/main/tutorial/tutorial_McpAdapters_DeepSeek_R1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCP with DeepSeek-R1 Tutorial

This notebook tutorial showcases a step-by-step guide on how to implement DeepSeek-R1 connected to tools in MCP servers, using LangChain's MCP Adapters library (here: https://github.com/langchain-ai/langchain-mcp-adapters).

I am using MCP servers from an MPC server registry/depository called MCP Server Cloud (here: https://mcpserver.cloud/, or their GitHub repo here: https://github.com/modelcontextprotocol).

I will be connecting DeepSeek-R1 to 2 MCP servers, with 1 tool in each MCP server. Namely, I will be using the Brave Search MCP Server (here: https://mcpserver.cloud/server/server-brave-search) and the AccuWeather MCP Server (here: https://mcpserver.cloud/server/mcp-weather-server).

To use the Brave Search MCP Server and the AccuWeather MCP Server, you will need to create a Brave Browser API key (here: https://brave.com/search/api/) and an AccuWeather API key (here: https://developer.accuweather.com/getting-started), respectively. They are both free and it's fairly straight forward to do this (but note creating a Brave Browser API key require a credit card even for the free subscription). Just ask any AI for the step-by-step guide to do this.

Once you have your Brave Browser and AccuWeather API keys, save them in a .env file, along with an OpenRouter API key (for this notebook tutorial I will be using DeepSeek-R1 hosted on OpenRouter). This .env file is saved in the same folder as where this Jupyter Notebook will be saved.

Now that we have all the above setup, let's get into the more technical part of this notebook tutorial. How LangChain's MCP Adapters library works is it convert tools in MCP servers into LangChain tools, so then these LangChain tools can be used within the LangChain/LangGraph framework. Yes, it's as simple as that!

Currently MCP servers are still in it's early development stages and so MCP servers doesn't yet have a direct SSE (Server-Sent Events) connection. To fix this, I have used a package called Supergateway (here: https://github.com/supercorp-ai/supergateway) which establishes a SSE connection for MCP servers. [Note: Currently there are several other ways to connect to MCP servers including downloading MCP servers into your local device and then connecting with the MCP server locally in your device using a Python package called langchain-mcp-tools (here: https://github.com/hideya/langchain-mcp-tools-py, where support for remote MCP server connection is currently experimental) or using the docker approach (here: https://www.youtube.com/watch?v=rdvt1qBZJtI), but I have chosen to use the Supergateway package approach as it is more realistic to connect to remote servers via SSE connections. The Supergateway package is run using npx (which is available in Node.js) which means if you haven't already, you will need to download Node.js (from here: https://nodejs.org/en/download) in order to use the Supergateway package via npx.]

Referring to the instructions in the README file in the Supergateway's GitHub repo, in particular the "stdio → SSE" section ("Expose an MCP stdio server as an SSE server:"):
- To establish a SSE connection for the Brave Search MCP Server using Supergateway, run the following command below in your IDE's (for eg. Cursor or VS Code) Terminal window (where this will use port 8001):
`npx -y supergateway --stdio "npx -y @modelcontextprotocol/server-brave-search" --port 8001 --baseUrl http://localhost:8001 --ssePath /sse --messagePath /message`
- To establish a SSE connection for the AccuWeather MCP Server using Supergateway, open a 2nd Terminal window in your IDE and run the following command below in this 2nd Terminal window (where this will use port 8002):
`npx -y supergateway --stdio "uvx --from git+https://github.com/adhikasp/mcp-weather.git mcp-weather" --port 8002 --baseUrl http://localhost:8002 --ssePath /sse --messagePath /message`

**Tip:** If you are unsure how to write the commands above for other MCP servers, just copy and paste the entire README file instructions in Supergateway's GitHub repo and the entire content of the MCP server page in the MCP Server Cloud registry/depository wesbite (for eg. for the Brave Search MCP Server, copy and paste the entire content from this page from the MCP Server Cloud registry/depository website: https://mcpserver.cloud/server/server-brave-search) into an AI and ask the AI to give you the "stdio → SSE" command.

Now that you have both the Brave Search MCP Server and AccuWeather MCP Server SSE connections running, you can now run the Python script below which uses `http://localhost:8001/sse` for the Brave Search MCP Server and `http://localhost:8002/sse` for the AccuWeather MCP Server.

Remember once you are done with using the MCP Servers, you can close off or disconnect the MCP Server's SSE connections by typing "CTRL" + "C" keys in your IDE's Terminal window.

**Takeaway:** This notebook tutorial demonstrates that even without having DeepSeek-R1 fine-tuned for tool calling or even without using my Tool-Ahead-of-Time package (since LangChain's MCP Adapters library works by converting tools in MCP servers into LangChain tools), MCP (via LangChain's MCP Adapters library) still works with DeepSeek-R1. This is likely because DeepSeek-R1 671B is a reasoning model and also how the prompts are written within LangChain's MCP Adapters library.

In [ ]:
# 下载官方最新LTS版（网页1方法优化）
!curl -fsSL https://nodejs.org/dist/v18.17.0/node-v18.17.0-linux-x64.tar.xz | tar -xJvf - -C /usr/local/

In [1]:
import os
# 将Node.js路径插入系统PATH首位（网页1[1](@ref)方法改进）
node_path = '/usr/local/node-v18.17.0-linux-x64/bin'
os.environ['PATH'] = f'{node_path}:{os.environ["PATH"]}'

In [ ]:
# 版本验证（网页5测试方法扩展）
print("Node.js版本：")
!node -v
print("\nnpm版本：")
!npm -v

# 功能测试（网页3补充方法）
!node -e "console.log('Node.js运行成功')"

Node.js版本：
v18.17.0

npm版本：
9.6.7
Node.js运行成功


In [ ]:
!nohup npx -y supergateway --stdio "uvx --from git+https://github.com/adhikasp/mcp-weather.git mcp-weather" --port 8002 --baseUrl http://localhost:8002 --ssePath /sse --messagePath /message > output.log 2>&1 &!

In [ ]:
!export OPENROUTER_API_KEY="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"  # 临时生效，重启会话后失效
!export ACCUWEATHER_API_KEY="GANBR3Tq1qcAj9AeU11H1IdPkxBe9OlM"  # 临时生效，重启会话后失效


In [ ]:
!pip install python-dotenv
!pip install pydantic
!pip install langchain-core
!pip install langchain-openai
!pip install langgraph
!pip install langchain-mcp-adapters

In [4]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
import nest_asyncio
import asyncio
from google.colab import userdata

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Load environment variable (ie. API key) from .env file
load_dotenv()



# 初始化硅基流动模型
model = ChatOpenAI(
    openai_api_key="sk-vthwrrguawekmhxxfiihtjanebombngupcgdmrwcngwpphms",
    base_url="https://api.siliconflow.cn/v1",  # 替换为实际 API 地址
    # model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"  # 模型名称
    model_name="deepseek-ai/DeepSeek-R1"
)


# Initialize model
# model = ChatOpenAI(
#     # model="deepseek/deepseek-r1",
#     # model="deepseek/deepseek-r1:free",
#     model="deepseek/deepseek-chat-v3-0324:free",
#     # api_key=os.environ["OPENROUTER_API_KEY"],
#     api_key=userdata.get("OPENROUTER_API_KEY1"),
#     base_url="https://openrouter.ai/api/v1"
# )

# Define the main asynchronous function
async def main():
    # Configure the MCP clients for Brave Search and Weather servers
    async with MultiServerMCPClient(
        {
            # "brave-search": {
            #     "url": "http://localhost:8001/sse",
            #     "transport": "sse",
            #     "headers": {
            #         "Authorization": os.environ["BRAVE_API_KEY"]  # Replace with your Brave Search API key
            #     }
            # },
            "weather": {
                "url": "http://127.0.0.1:8002/sse",
                "transport": "sse",
                "headers": {
                    "Authorization": userdata.get("ACCUWEATHER_API_KEY")  # Replace with your AccuWeather API key
                }
            }
        }
    ) as client:
        # Create the agent with access to the tools provided by the MCP servers
        agent = create_react_agent(model, client.get_tools())

        # Example usage: Perform a web search using Brave Search
        # search_response = await agent.ainvoke({
        #     "messages": "Search for the latest news on AI."
        # })
        # print(search_response['messages'][-1].content)

        # Example usage: Get the weather forecast using the Weather MCP Server
        weather_response = await agent.ainvoke({
            "messages": "明天郑州的天气怎么样?"
        })
        print(weather_response['messages'][-1].content)
        print(weather_response)

# Run the main function using asyncio.ensure_future
# asyncio.ensure_future(main())
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(main())


<Task pending name='Task-5' coro=<main() running at <ipython-input-4-04d625081f1e>:39>>

In [ ]:
!pip install langchain_community